In [1]:
from langchain_community.document_loaders import TextLoader # loading txt file 
import pypdf # loading pdf files 

In [2]:
file_pdf = "ARYAN PAHARI.pdf" 
file_txt = "example.txt"

## TxT file extractor

In [3]:
loader = TextLoader(file_txt)
document = loader.load()
page_inside_content = document

In [4]:
txt_document_content = "" 
for page in page_inside_content:
    txt_document_content += page.page_content

## PDF file extractor

In [5]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(file_pdf)
pdf_document = loader.load()
# pages = []
# async for page in loader.alazy_load(): 
#     pages.append(page)

# RAG immplementation for QnA using Open Source LLM and AstraDB

##### Creating Embeddings 

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 20)
content = text_splitter.split_documents(pdf_document)

In [7]:
import os
from dotenv import load_dotenv
load_dotenv() 
api_key = os.environ.get("GROQ_API_KEY")
cohere_api_key = os.environ.get("COHERE_API_KEY")

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1")

d:\My File\Internship\AryanPahari_Task_4\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents=content , embedding=embeddings)

In [10]:
retriver = db.as_retriever(k=min(len(db), 10))

In [11]:
retriver.vectorstore.embeddings

HuggingFaceEmbeddings(model_name='multi-qa-MiniLM-L6-cos-v1', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
len(db)

6

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_groq import ChatGroq

In [14]:
api_key

'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'

In [15]:

llm = ChatGroq(
    model="gemma-7b-it",
    temperature=0.7,
    max_retries=2,
    api_key=api_key
)

In [16]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
<context>
Question: {input}
""")

In [17]:
chain = create_stuff_documents_chain(llm=llm,prompt=prompt)

In [18]:
retriver_chain = create_retrieval_chain(retriver,chain) 

In [19]:
answer = retriver_chain.invoke({"input" : "what projects he had made ?"})

In [20]:
from IPython.display import Markdown, display

display(Markdown(answer["answer"]))

The provided text lists several projects the individual has worked on, including:

- **Postmedia:** A beginner social media app featuring user profiles, post likes, and search functionality.


- **Hotel Management GUI:** A GUI-based hotel management system with features for room booking, availability checks, post-checkout, and customer reviews.


- **AgriTech Trailblazers:** A tool for crop suggestions, mixed cropping, and accessing government subsidies.


- **Holographic Microplastic Image Classification:** A Streamlit app using CNN to classify types of microplastics in holographic images.


- **Smart Agricultural Monitoring:** An ML-powered app for farmers, providing crop recommendations.